<a href="https://www.kaggle.com/code/amirmotefaker/ukraine-russia-war-twitter-sentiment-analysis?scriptVersionId=127550653" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

# Introduction

- More than 400 days have passed since the war between Russia and Ukraine. Many countries support Ukraine by imposing economic sanctions against Russia. There are a lot of tweets about the Ukraine-Russia war where people tend to update the facts on the ground, how they feel about it, and who they support.

# Russia-Ukraine war at a glance: what we know on day 429 of the invasion

- Russia on Friday launched a wave of missile attacks across many of Ukraine’s biggest cities, killing a mother and young child in the port city of Dnipro, and three people at a high-rise apartment building in the central city of Uman. Air raid alarms were active across the country in the early hours of Friday morning, while explosions were heard in Kyiv, and southern Mykolaiv was targeted again.

- At least seven civilians were killed and 33 injured between Wednesday and Thursday, Ukraine’s presidential office said, including one person killed and 23 wounded when four Kalibr cruise missiles hit the southern city of Mykolaiv.

- The parliamentary assembly of the Council of Europe has voted that the forced detention and deportation of children from Russian occupied territories of Ukraine is genocide.

- [The Guardian](https://www.theguardian.com/world/2023/apr/28/russia-ukraine-war-at-a-glance-what-we-know-on-day-429-of-the-invasion)


# Import Libraries

In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator
import nltk
import re
from nltk.corpus import stopwords
import string

/opt/conda/lib/python3.7/site-packages/nltk/twitter/__init__.py:20: UserWarning: The twython library has not been installed. Some functionality from the twitter package will not be available.
  warnings.warn("The twython library has not been installed. "


# Read Data

In [2]:
data = pd.read_csv("/kaggle/input/russia-vs-ukraine-tweets-datasetdaily-updated/filename.csv")